In [ ]:
import json
import csv
import shapely
import pandas as pd

from typing import TypedDict, Union, Required

class Coordinates(TypedDict):
    lat: float
    lon: float

class Team(TypedDict, total=False):
    name: str
    venue: str
    L: float
    S: float
    state: Union[str, list[str]]
    coordinates: Coordinates

class League(TypedDict):
    league_name: str
    teams: list[Team]

def load_counties_data():
    with open("./counties.geojson") as f:
        counties_geojson = json.load(f)
      
    co_data_frame = pd.read_csv('./co-est2024-alldata.csv', encoding='iso-8859-1')

    for feature in counties_geojson["features"]:
        coords_stack = [ feature["geometry"]["coordinates"] ]

        while isinstance(coords_stack[-1][0], list):
            coords_stack.append(coords_stack[-1][0]) 

        coords = coords_stack[-2]    
        try: 
            raw_polygon = shapely.geometry.Polygon(coords) 
            state = feature["properties"]["STATEFP"]
            county = feature["properties"]["COUNTYFP"]
            co_data_frame.loc[co_data_frame.query(f"STATE == {state} and  COUNTY == {county}").index, "centroid"] = raw_polygon.centroid
        except Exception as e:
            print(e)
            print("Invalid coordinates:", feature["properties"]["Name"])  
    return (counties_geojson, co_data_frame)

(counties_geojson, co_data_frame) = load_counties_data()


In [ ]:
import numpy as np
import pandas as pd
import json

def haversine_miles(centroid, lat2, lon2):
    r = 3958.8  # earth radius miles
    lat1 = centroid.y
    lon1 = centroid.x
    phi1 = np.radians(lat1); phi2 = np.radians(lat2)
    dphi = phi2 - phi1
    dlambda = np.radians(lon2 - lon1)
    a = np.sin(dphi/2.0)**2 + np.cos(phi1)*np.cos(phi2)*np.sin(dlambda/2.0)**2
    return 2*r*np.arcsin(np.sqrt(a))

leagues = {
    'NFL': {
        "weight": 0.9
    },
    'NBA': {
        "weight": 0.6
    },
    'MLB': {
        "weight": 0.5
    },
    'NHL': {
        "weight": 0.25
    }, 
    'MLS': {
        "weight":0.15
    }
}    

leagues_included = [ "MLB", "NHL", "NFL", "NBA", "MLS" ]

for league in leagues_included:
  with open(f"./teams_{league}.json") as f:
    leagues[league]["json"]: League = json.load(f)

    teams = pd.DataFrame(leagues[league]["json"]["teams"])

    teams['S'].fillna(0.0, inplace=True)
    teams['L'].fillna(5.0, inplace=True)
       
    # compute distance matrix
    d = np.zeros((len(co_data_frame), len(teams)))
    for i, c in co_data_frame.iterrows():
        if not pd.isna(c["centroid"]):
            for j, t in teams.iterrows():
                d[i,j] = haversine_miles(c["centroid"], t["coordinates"]['lat'], t["coordinates"]['lon'])


    leagues[league]["teams"] = teams   
    leagues[league]["distances"] = d  

In [ ]:
import numpy as np
import pandas as pd

distance_decay = 0.005 
competition_temperature = 0.5  
not_nearest_multiplier = 2.0 # added to distance multiplied (d - nearest_d) 
not_same_state_multiplier = 2.0 # TODO might need bigger multipler if different country, eh

for league in leagues_included: 
    d = leagues[league]["distances"] 
    
    # compute raw R_{c,t}
    nearest_key = "nearest"
    R = np.zeros_like(d)
    league_weight = leagues[league]["weight"]

    co_data_frame[nearest_key] = float('nan')
    for j, t in leagues[league]["teams"].iterrows():
        for i, c in co_data_frame.iterrows():
            nearest = co_data_frame.loc[i, nearest_key]
            if pd.isna(nearest) or d[i,j] < nearest:
                co_data_frame.loc[i, nearest_key] = d[i,j] 
    
    for j, t in leagues[league]["teams"].iterrows():
        for i, c in co_data_frame.iterrows():
            effective_d = d[i,j]
            nearest_d = c[nearest_key]
            if not pd.isna(nearest_d) and effective_d > nearest_d:
                effective_d = nearest_d + ((effective_d - nearest_d) * not_nearest_multiplier)
            team_state = t["state"] 
            county_state = c["STNAME"]
            if isinstance(team_state, list):
                if not county_state in team_state:
                    effective_d *= not_same_state_multiplier 
            elif county_state != team_state:
                effective_d *= not_same_state_multiplier      
            team_distance_decay = distance_decay  # TODO?
            D = np.exp(-team_distance_decay * effective_d) 
            R[i,j] = (league_weight + (t["L"] + t["S"]) / 5) * D   

    expR = np.exp(R / competition_temperature)
    leagues[league]["shares"] =  expR / expR.sum(axis=1, keepdims=True)    
    leagues[league]["Rs"] = R
  

In [ ]:
for league in leagues_included: 
    d = leagues[league]["distances"]
    R = leagues[league]["Rs"]
    
    shares = leagues[league]["shares"]
    # fans = (shares * co_data_frame['POPESTIMATE2020'].values.reshape(-1,1)).astype(int)
    
    df_out = []
    for i, c in co_data_frame.iterrows():
        for j, t in leagues[league]["teams"].iterrows():
            if shares[i,j] > 0.035:
                df_out.append({
                    'county': c['CTYNAME'],
                    'countyfp': c['COUNTY'],
                    'state': c['STNAME'],
                    'statefp': c['STATE'],
                    'team_name': t['name'],
                    'population': c['POPESTIMATE2020'],
                    # 'distance_miles': round(d[i,j],1),
                    # 'raw_R': R[i,j],
                    'share': shares[i,j],
                    # 'fans': fans[i,j]
            })
    leagues[league]["dfs"] = pd.DataFrame(df_out)

In [ ]:
default_style = {
        "color": "grey",
        "weight": 1,
        "fillColor": "grey",
        "fillOpacity": 0.1,
    }

def opactity_for_population(population): 
    if population > 5000000:
        return 0.9
    if population > 2000000:
        return 0.8
    if population > 1000000:
        return 0.7  
    if population > 500000:
        return 0.6 
    if population > 100000:
        return 0.5 
    if population > 50000:
        return 0.4
    if population > 10000:
        return 0.3
    if population > 5000:
        return 0.2
    return 0.1

for feature in counties_geojson["features"]:
    feature["properties"]["style"] = default_style
                

def highlight(league, team_color_map, share_theshold = 0.05): 
    for feature in counties_geojson["features"]:
       state = feature["properties"]["STATEFP"]
       county = feature["properties"]["COUNTYFP"]
       county_rows = leagues[league]["dfs"].query(f"statefp == {state} & countyfp == {county}").sort_values(by="share", ascending=False)
       if county_rows.shape[0] > 0:
           group_cols = list(county_rows.columns.difference(['share']))
           county_rows = county_rows.groupby(group_cols, as_index=False).max().sort_values(by="share", ascending=False)
           county_row = county_rows.iloc[0]
           if isinstance(team_color_map, dict) and county_row["team_name"] in team_color_map.keys() and county_row['share'] > share_theshold:   
                feature["properties"]["style"] = {
                "color": "grey",
                "weight": 1,
                "fillColor": team_color_map[county_row["team_name"]],
                "fillOpacity": opactity_for_population(county_row["population"] * county_row["share"])
                }               
           elif isinstance(team_color_map, str):
               feature["properties"]["style"] = {
                "color": "grey",
                "weight": 1,
                "fillColor": team_color_map,
                "fillOpacity": opactity_for_population(county_row["population"])
               }   
            
#highlight("NHL", "green")

nhl_color_map = { 
    "St. Louis Blues": "blue",
    "Chicago Blackhawks": "black",
    "Minnesota Wild": "darkgreen",
    "Nashville Predators": "gold",
    "Detroit Red Wings": "red",
    "Dallas Stars": "limegreen",
    "Colorado Avalanche": "maroon",
    "Columbus Blue Jackets": "midnightblue",
    "Pittsburgh Penguins": "gold",
    "Philadelphia Flyers": "darkorange",
    "Washington Capitals": "navy",
    "New York Rangers": "red",
    "New York Islanders": "mediumblue",
    "New Jersey Devils": "black",
    "Buffalo Sabres": "navy",
    "Boston Bruins": "gold",
    "San Jose Sharks": "teal",
    "Utah Mammoth": "skyblue",
    "Los Angeles Kings": "black",
    "Anaheim Ducks": "orangered",
    "Seattle Kraken": "aquamarine",
    "Vegas Golden Knights": "goldenrod",
    "Tampa Bay Lightning": "navy",
    "Florida Panthers": "crimson",
    "Carolina Hurricanes": "red"
}

mlb_color_map = {
    "St. Louis Cardinals": "red",
    "Chicago Cubs": "blue",
    "Chicago White Sox": "black",
    "Kansas City Royals": "skyblue",
    "New York Yankees": "black",
    "Boston Red Sox": "firebrick",
    "Athletics": "yellow"
}

#highlight("NHL", nhl_color_map )
#highlight("MLB", mlb_color_map )

In [ ]:
import ipywidgets as widgets
import pandas as pd

from ipyleaflet import Map, Marker, MarkerCluster, GeoJSON, Popup, FullScreenControl, basemaps
from shapely.geometry import Polygon, Point

map = Map(basemap=basemaps.CartoDB.Positron, center=[38.72728229549864, -96.9010842308538], zoom=5, scroll_wheel_zoom=True)

def show_teams(event, feature, **kwargs): 
    statefp = feature["properties"]["STATEFP"]
    countyfp = feature["properties"]["COUNTYFP"]
    row = co_data_frame.query(f"STATE == {statefp} & COUNTY == {countyfp}") 
    centroid = row["centroid"].iloc[0]
    population = row["POPESTIMATE2020"].iloc[0]

    all_county_rows = pd.DataFrame()    
    leagues_table = ""  
    try:
        for league in leagues_included:
            county_rows = leagues[league]["dfs"].query(f"statefp == {statefp} & countyfp == {countyfp}")
            group_cols = list(county_rows.columns.difference(['share']))
            county_rows = county_rows.groupby(group_cols, as_index=False).max()
            county_rows["league"] = league
            all_county_rows = pd.concat([all_county_rows, county_rows], ignore_index=True)
      

        all_county_rows = all_county_rows.sort_values(by="share", ascending=False)
        for i, county_row in all_county_rows.iterrows():
            if county_row['share'] > 0.05:
                leagues_table += (
                 "<tr>"  
                 f"<td>{county_row['league']}</td>" 
                 f"<td>{county_row['team_name']}</td>" 
                 f"<td>{round(county_row['share'] * 100, 1)}%</td>" 
                 # f"<td>{county_row['fans']}</td>" 
                 "</tr>")
       
        popup = Popup(location=(centroid.y, centroid.x), 
                  child=widgets.HTML("<table border='1' style='border-collapse: collapse'>" +
                  f'<caption>{feature["properties"]["Name"]} - {population}</caption>' +    
                  leagues_table +
                  "</table>"  
        )
        )
    except Exception as e:
        popup = Popup(location=(centroid.y, centroid.x), 
                  child=widgets.HTML(str(e)))
    map.add(popup)    
    

layer = GeoJSON(data = counties_geojson, 
    hover_style = {"fillColor": "white"}
)

layer.on_click(show_teams)

map.add(layer)

map.add(FullScreenControl())


In [ ]:
map.fullscreen = True

In [ ]:
pd.set_option('display.float_format', lambda x: '%.0f' % x)
for league in leagues_included:
    league_dfs = leagues[league]["dfs"]
    league_dfs["share_of_pop"] = league_dfs['share'] * league_dfs["population"]
    group_cols = list(league_dfs.columns.difference(['team_name', 'share_of_pop']))
    print(league_dfs[['team_name', 'share_of_pop']].groupby('team_name').sum().sort_values(by='share_of_pop',ascending=False)) #.sort_values(by=['team_name'])